In [5]:
import urllib
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime
import calendar
#import scrapy
import re
from bson.objectid import ObjectId









In [6]:
import pymongo
def parseEventPage(month, year):
    #Download the Page Content
#     URL = 'https://en.wikipedia.org/wiki/Portal:Current_events/%s_%s' %(month,year)
    URL = 'https://en.wikipedia.org/wiki/Portal:Current_events/%s_%s' %(month,year)
    data = urllib.request.urlopen(URL).read()
    soup = BeautifulSoup(data,'html.parser')
    #soup = BeautifulSoup(data,'lxml') 
#     print(soup)
    

    #Selecting the Title
    body = soup.find('body')
    
#     print(body)

    ## define a mongodb client, create a database names WikiEvent
    client = pymongo.MongoClient("localhost", 27017)
    #db = client['datacampdb']
#     myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# mydb = myclient["mydatabase"]
    db = client['WikiEvent']
#     print(client.list_database_names())
    serverStatusResult=db.command("serverStatus")
#     print(serverStatusResult)
#     print(db)
    check=body.find_all('tr')[1]
    print(check)
    
    ## extract date for events
    datetables = body.find_all('table', attrs={'class':'plainlinks'})
#     print (datetables)
    print ('Are you here:')
    ## events on each day identified by td tag with description class attribute
    for date, dailyevent in zip(datetables, body.find_all('td', attrs={'class':'description'})):
        print ('Hi, are you there')
        datenumber = datetime.strptime(re.sub('\s*','',date.find('span', attrs={'class':'bday dtstart published updated'}).text), '%Y-%m-%d')
        ##categories identified with dt tag
        categories = dailyevent.find_all('dt')
        
        print (categories)
        print ('date: %s' %datenumber)
         #double equal loop, category and concrete events
        for category,eventcategory in zip(categories,dailyevent.find_all('ul', recursive = False)):
            categorytext = category.text
            print ('category: %s' %categorytext)
            ## single event identified with li tag. if event has specific event entity page, there will be one more li tag
            for singleevent in eventcategory.find_all('li', recursive=False):
                ## event has wikipage
                if singleevent.find('ul'):

                    singles = singleevent.find_all('ul')
                   
                    singleeventsummary = singles.pop().text  ## last li tag, event summary

                    ## event title may have multiple subtitles

                    title = [st.get('href') for st in singleevent.find_all('a', recursive=False)]

                    ## subtitle exists
                    if singles:

                        title = title + [single.find('a').get('href') for single in singles]


                    entities = [entitylink.get('href') for entitylink in singleevent.find('ul').find('li').find_all('a') if entitylink.get('title')]
                    external = singleevent.find_all('a', attrs={'class': 'external text'})
                    external_link = [e.get('href') for e in external if external]

                    print ('event title: ' )
                    print (title)
                
                else:
                    title = False
                    singleeventsummary = singleevent.text
                    entities = [entitylink.get('title') for entitylink in singleevent.find_all('a') if entitylink.get('title')]
                    
                    external = singleevent.find_all('a', attrs={'class':'external text'})
                    external_link = [e.get('href') for e in external if external]
                print ('event summary: %s' %singleeventsummary)
                myrecord={"date": datenumber,
                          "category": categorytext,
                          "event_title": title,
                          "event_summary": singleeventsummary,
                          "entities": entities,
			              "person": False,
			              "external_link": external_link
                         }
                db.events_entity_title.insert_one(myrecord)
                
#                     print(myrecord)
                #document = client.db.collection.find_one({'_id': ObjectId(myrecord)})
                 




In [7]:


import urllib.request

In [8]:
# if __name__ == '__main__':

#     #body = getEventPage('January', '2017')
#     y = '2017'
#     m = 'January'
    
#     parseEventPage(m,y)
                


In [9]:
if __name__ == '__main__':

    #body = getEventPage('January', '2017')
    year = ['2015','2016','2017','2018','2019']
    month = [calendar.month_name[i].capitalize()for i in range(1,13)]
    print(month)
    month1= [calendar.month_name[i].capitalize()for i in range(1,5)]
    #month=month.capitalize()
    
    for y in year:
            if y!='2019':
                for m in month:
                    print (y,m)
                    parseEventPage(m,y)
            elif y=='2019':
                for m in month1:
                    print (y,m)
                    parseEventPage(m,y)
                


['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
2015 January
<tr>
<td>
<table class="plainlinks" role="presentation" style="display:table; width:100%; background-color:#cef2e0; border-spacing:0; font-weight:bold;box-sizing:border-box;">
<tbody><tr>
<td style="text-align:left;font-size:11pt; padding:5px; margin-left:0px; width:100%"><span class="summary"><span style="display:none;">Current events of </span>January 1, 2015<span style="display:none"> (<span class="bday dtstart published updated">2015-01-01</span>)</span> (Thursday)</span>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="https://en.wikipedia.org/w/index.php?title=Portal:Current_events/2015_January_1&amp;action=edit&amp;editintro=Portal:Current_events/Edit_instructions">edit</a></small>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink

Hi, are you there
[<dt>Armed conflicts and attacks</dt>, <dt>Business and economy</dt>, <dt>Disasters and accidents</dt>, <dt>Law and crime</dt>, <dt>Politics and elections</dt>, <dt>Science and technology</dt>, <dt>Sports</dt>]
date: 2015-01-09 00:00:00
category: Armed conflicts and attacks
event title: 
['/wiki/Charlie_Hebdo_shooting', '/wiki/France', 'https://www.cnbc.com/id/102323454']
event summary: The concurrent standoff ends with Coulibaly dead as well as four hostages. Four additional hostages and two police officers require hospitalization. Boumeddiene remains at large. (MSNBC)
event title: 
['/wiki/2015_Baga_massacre']
event summary: Refugees flee Nigeria's Borno State following the Boko Haram massacre in the town of Baga. 7,300 flee to neighbouring Chad while over 1,000 are trapped on the island of Kangala in Lake Chad. Nigeria's army vows to recapture the town, while Niger and Chad withdraw their forces from a transnational force tasked with combating militants. (UNHCR) (N

event summary: According to preliminary tests at the Glendive, Montana city water treatment plant, the January 17 oil spill in the Yellowstone River contains benzene, a chemical that causes cancer. (National Geographic)(LA Times)
category: Law and crime
event summary: A gunman shoots a cardiovascular surgeon to death at Harvard Medical School's Brigham and Women's Hospital in Boston, Massachusetts, and then commits suicide. (USA Today)
category: Politics and elections
event title: 
['/wiki/Zambian_presidential_election,_2015']
event summary: Voters in Zambia go to the polls for a presidential election following the October 2014 death of incumbent President Michael Sata. Defence Minister Edgar Lungu (the Patriotic Front) and Hakainde Hichilema (the United Party for National Development) seek the office. (AFP via Yahoo! News)
event title: 
['/wiki/2015_State_of_the_Union_Address']
event summary: The President of the United States Barack Obama delivers a State of the Union address to the 

<tr>
<td>
<table class="plainlinks" role="presentation" style="display:table; width:100%; background-color:#cef2e0; border-spacing:0; font-weight:bold;box-sizing:border-box;">
<tbody><tr>
<td style="text-align:left;font-size:11pt; padding:5px; margin-left:0px; width:100%"><span class="summary"><span style="display:none;">Current events of </span>February 1, 2015<span style="display:none"> (<span class="bday dtstart published updated">2015-02-01</span>)</span> (Sunday)</span>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="https://en.wikipedia.org/w/index.php?title=Portal:Current_events/2015_February_1&amp;action=edit&amp;editintro=Portal:Current_events/Edit_instructions">edit</a></small>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="https://en.wikipedia.org/w/index.php?title=Portal:Current_events/2015_February_1&amp;action=his

event summary: At the 65th Berlin International Film Festival, the film Taxi by Iranian director Jafar Panahi wins the Golden Bear. (BBC)
event summary: French film and television actor Louis Jourdan (Gigi) dies at the age of 93. (People)
category: Disasters and accidents
event summary: There is a blizzard warning in place for eastern Massachusetts in the northeastern United States. (Boston.com)
category: Health
event summary: Twelve people die of swine flu in the Indian state of Gujarat taking the death toll to 136 since January with 130 new cases reported. (IBN live)
category: International relations
event title: 
['/wiki/Yemeni_Crisis_(2011%E2%80%93present)']
event summary: The United Arab Emirates suspends its embassy operations in Yemen due to security concerns surrounding the recent Shiite Houthi rebel take over of the country. (AFP via France 24)
category: Law and crime
event summary: The Prime Minister of Australia Tony Abbott issues a last-minute plea for Bali Nine drug smuggl

...

event title: 
['/wiki/Crime_in_California']
event summary: A shooting at a motel in Sacramento County, California, leaves a Sacramento County Sheriff's Department deputy dead and two California Highway Patrol officers injured. The suspect is later arrested. (NBC News) (The Sacramento Bee)
category: Science and technology
event title: 
['/wiki/Fast_radio_burst']
event summary: Researchers at the Breakthrough Listen initiative announce they have detected 15 new powerful fast radio bursts (FRB) from FRB 121102, located in a dwarf galaxy about three billion light-years from Earth. FRB 121102 was first detected in 2012, and gained further attention after FRBs were detected from there again in 2015. (New Scientist) (Phys)
Hi, are you there
[<dt>Business and economy</dt>, <dt>Disasters and accidents</dt>, <dt>International relations</dt>, <dt>Law and crime</dt>, <dt>Politics and elections</dt>]
date: 2017-08-31 00:00:00
category: Business and economy
event title: 
['/wiki/Wells_Fargo_account_

event title: 
['/wiki/List_of_accidents_and_incidents_involving_helicopters']
event summary: A Duke University medical helicopter crashes near Belvidere, North Carolina, killing four people. (Daily Mail)
A helicopter crashes in Medford, New Jersey, leaving two people dead, including singer Troy Gentry of Montgomery Gentry. (Sky News)
category: International relations
event title: 
['/wiki/Bahrain%E2%80%93United_States_relations']
event summary: The U.S. State Department has approved a package of potential arms sales worth more than USD$ 3.8 billion to the Kingdom of Bahrain including F-16 jets, upgrades, missiles and patrol boats. (Reuters)
event title: 
['/wiki/North_Korea%E2%80%93Philippines_relations']
event summary: The Philippines suspends trade relations with North Korea to comply with a recent United Nations Security Council resolution over its repeated missile tests. (Reuters)
event title: 
['/wiki/2017_Qatar_diplomatic_crisis']
event summary: Qatar Emir Sheikh Tamim bin Hamad 

event title: 
['/wiki/Next_Japanese_general_election']
event summary: Japanese Prime Minister Shinzō Abe is expected to dissolve the House of Representatives on September 28, 2017 to make way for fresh elections. (The Japan Times)
event title: 
['/wiki/Next_Palestinian_general_election']
event summary: Hamas, through Egyptian-mediated talks, agrees to dissolve its Gaza administration, hold talks with the Fatah West Bank government, and hold elections for a Palestinian unity government. (The Guardian) (Reuters)
category: Sports
event title: 
['/wiki/EuroBasket_2017']
event summary: In basketball, Slovenia wins EuroBasket for the first time in history, beating Serbia 93–85 in the final. (EuroNews)
event title: 
['/wiki/2017_Asian_Indoor_and_Martial_Arts_Games']
event summary: The fifth Asian Indoor Games open in Ashgabat, Turkmenistan, with around 5,500 athletes from 61 countries participating in the event which will last until September 27. (Radio Free Europe/Radio Liberty)
Hi, are you 

event title: 
['/wiki/Politics_of_Uzbekistan']
event summary: Uzbekistan dissident writer Nurulloh Muhammad Raufkhon is arrested on his return to the Central Asian country from exile. (Reuters)
event title: 
['/wiki/Politics_of_Thailand']
event summary: Former Thai Prime Minister Yingluck Shinawatra is sentenced to five years in prison in absentia. (BBC)
category: Politics and elections
event title: 
['/wiki/Japanese_general_election,_2017']
event summary: Tokyo governor Yuriko Koike establishes the conservative Kibō no Tō (Party of Hope) to contest the upcoming election although she will neither lead it nor run. (Kyodo)
event title: 
['/wiki/Abortion_in_the_Republic_of_Ireland']
event summary: The Republic of Ireland announces a referendum on abortion will be held between May and June 2018. (The Telegraph)
event title: 
['/wiki/Iraqi_Kurdistan_independence_referendum,_2017']
event summary: The government of Iraqi Kurdistan announces that they won the referendum by over 92% of the vote

<tr><th style="text-align:center;width:14%">S</th><th style="text-align:center;width:14%">M</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">W</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">F</th><th style="text-align:center;width:14%">S</th></tr>
Are you here:
2017 November
<tr><th style="text-align:center;width:14%">S</th><th style="text-align:center;width:14%">M</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">W</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">F</th><th style="text-align:center;width:14%">S</th></tr>
Are you here:
2017 December
<tr><th style="text-align:center;width:14%">S</th><th style="text-align:center;width:14%">M</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">W</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">F</th>